In [ ]:
import phoenix as px
import pandas as pd
import numpy as np


# Launch phoenix
session = px.launch_app()


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
# The API key for your Azure OpenAI resource.
api_key = os.getenv("AZURE_OPENAI_API_KEY")
# Setting up the deployment name
gpt_model_name = os.getenv("AZURE_OPENAI_GPT_MODEL")
embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
# Currently Chat Completions API have the following versions available: 2023-03-15-preview
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_type = "azure"

In [ ]:
# Once you have started a Phoenix server, you can start your LangChain application with the OpenInferenceTracer as a callback. To do this, you will have to instrument your LangChain application with the tracer:

from phoenix.trace.langchain import LangChainInstrumentor

# By default, the traces will be exported to the locally running Phoenix server.
LangChainInstrumentor().instrument()

# Initialize your LangChain application
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain.retrievers import KNNRetriever

In [ ]:
embeddings = AzureOpenAIEmbeddings(
    model=embedding_model_name,
    api_version=api_version,
)
documents_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/langchain-pinecone/database.parquet"
)
knn_retriever = KNNRetriever(
    index=np.stack(documents_df["text_vector"]),
    texts=documents_df["text"].tolist(),
    embeddings=embeddings,
)
chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank

llm = AzureChatOpenAI(
    model=gpt_model_name,
    api_version=api_version,
    )

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=knn_retriever,
)

# Instrument the execution of the runs with the tracer. By default the tracer uses an HTTPExporter
query = "What is euclidean distance?"
response = chain.invoke(query)

# By adding the tracer to the callbacks of LangChain, we've created a one-way data connection between your LLM application and Phoenix.

# To view the traces in Phoenix, simply open the UI in your browser.
session.url